In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation, Concatenate, Lambda
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers, activations
from tensorflow.keras.regularizers import l2,l1,l1_l2
import cv2
import os
from sklearn.utils import shuffle
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.applications.inception_resnet_v2 import InceptionResNetV2
print("Done Importing")

Done Importing


In [ ]:
def normalize(data):
        pixels=np.array(data)
        mean=pixels.mean()
        return pixels-mean

datagen=ImageDataGenerator( featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=360,
    width_shift_range=[10,40],
    height_shift_range=[10,40],
    brightness_range=None,
    shear_range=0.2,
    zoom_range=[0,0.2],
    channel_shift_range=0.0,
    fill_mode="nearest",
    cval=0.0,
    horizontal_flip=True,
    vertical_flip=True,
    rescale=None,
    # preprocessing_function=cv2.normalize,
    data_format=None,
    validation_split=0.2,
    dtype=None,)

img_dir='/content/drive/MyDrive/DATASET'

train_generator=datagen.flow_from_directory( img_dir,target_size=(300,300),batch_size=32,subset='training')
validation_generator=datagen.flow_from_directory(img_dir,target_size=(300,300),subset='validation')


Found 7203 images belonging to 24 classes.
Found 1790 images belonging to 24 classes.


In [ ]:
train_generator.class_indices

{'Basal Cell Carcinoma': 0,
 'Foot corn': 1,
 'Lyme Disease': 2,
 'Psoriasis': 3,
 'acne vulgaris': 4,
 'actinic cheilitis': 5,
 'alopecia': 6,
 'boils': 7,
 'bullous pemphigoid': 8,
 'dermatofibroma': 9,
 'eczema asteatotic': 10,
 'flea_bites': 11,
 'flexural psoriasis': 12,
 'hives': 13,
 'keloid': 14,
 'kerion': 15,
 'melanocytic nevi ': 16,
 'nail_melanoma': 17,
 'prurigo_nodularis': 18,
 'rosacea': 19,
 'scabies': 20,
 'seborrheic keratosis': 21,
 'straie': 22,
 'vitiligo': 23}

In [ ]:
# ir2=InceptionResNetV2(include_top=False, input_shape=(300, 300, 3))
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications import VGG19

# ir2 = VGG19(input_shape = (300, 300, 3), include_top = False, weights = 'imagenet')
ir2=tf.keras.applications.EfficientNetB4(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(300,300,3),
    pooling=None,
)
print("Model Imported!")

Model Imported!


In [ ]:
output = ir2.layers[-1].output
output = keras.layers.Flatten()(output)
ir2_model = Model(ir2.input, output)

In [ ]:
model=Sequential()
model.add(ir2_model)

In [ ]:
for layers in model.layers:
  layers.trainable=False

In [ ]:
from keras import backend as K

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


In [ ]:
# model = keras.models.load_model('/content/drive/MyDrive/model/EffectiveNetB4_80.h5')


In [ ]:
# model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.01),bias_regularizer=l2(0.06),input_dim=ir2_model.output_shape[1]))

model.add(Dense(128, activation='relu',input_dim=ir2_model.output_shape[1]))

# model.add(Dense(256, activation='relu'))
# # model.add(Dense(256, activation='relu'))
# model.add(Dense(128, activation='relu'))
# # model.add(Dropout(0.4))
# model.add(Dense(64, activation='relu'))
# # model.add(Dropout(0.2))


model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
# model.add(Dense(32, activation='relu'))
# model.add(Dropout(0.1))

model.add(Dense(24, activation='softmax'))

lr_schedule=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.1,decay_steps=10000,decay_rate=0.9)
opt=keras.optimizers.SGD(learning_rate=lr_schedule)

model.compile(loss="categorical_crossentropy",metrics=['accuracy','top_k_categorical_accuracy'],optimizer='adam')


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model (Functional)           (None, 179200)            17673823  
_________________________________________________________________
dense (Dense)                (None, 256)               45875456  
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2

In [ ]:
early_stopping=tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=10, verbose=0,
    mode='auto', baseline=None, restore_best_weights=True
)
 
results=model.fit_generator(train_generator,epochs=1,validation_data=validation_generator)


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


129/451 [=======>......................] - ETA: 19:19 - loss: 4.5408 - accuracy: 0.0780 - top_k_categorical_accuracy: 0.3208

KeyboardInterrupt: ignored

In [ ]:
# model.save_weights("/content/drive/MyDrive/IV3/weights.h5")

In [ ]:
history=results
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
history=results
plt.plot(history.history['top_k_categorical_accuracy'])
plt.plot(history.history['val_top_k_categorical_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
test_steps_per_epoch = np.math.ceil(validation_generator.samples / validation_generator.batch_size)
# print(test_steps_per_epoch)
predictions = model.predict_generator(validation_generator, steps=test_steps_per_epoch)
# Get most likely class
predicted_classes = numpy.argmax(predictions, axis=1)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2001: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


KeyboardInterrupt: ignored

In [ ]:
true_classes = validation_generator.classes
class_labels = list(validation_generator.class_indices.keys())

In [ ]:
import sklearn.metrics as metrics
report = metrics.classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report)    

NameError: ignored